In [ ]:
import numpy as np
import pandas as pd
from multiprocess import Pool

from benchmarking.multiprocessing.functions import evaluate_ar, evaluate_multiplier_iid, evaluate_ma
from bstrapping.synthetic_time_series.magarch11 import MovingAverageGARCH11

In [ ]:
benchmark = []
means_of_variance = []
std_of_variance = []
coverage_probability = []

list_name_weights = ['AR',
                     'Multiplier',
                     'MA',
                     ]

In [ ]:
sample_size = 10000
runs = 250

mean = 4
alpha0 = 1
alpha = 0.25
beta = 0.25
parameters = np.array([0.5 ** i for i in range(1, 3)])

In [ ]:
samples = [
    MovingAverageGARCH11(mean=mean,parameters=parameters, alpha0=alpha0, alpha=alpha, beta=beta).generate_samples(sample_size)
    for _ in range(runs)]

In [ ]:
time_series = MovingAverageGARCH11(mean=mean,parameters=parameters, alpha0=alpha0, alpha=alpha, beta=beta)
time_series.asymptotic_variance

## Benchmark bootstraps

In [ ]:
%%time
%%capture
# benchmark AR bootstrap
p = Pool()

evaluations_ar = p.map(lambda sample: evaluate_ar(sample, alpha=0.1, mean=mean), samples)

result = np.array(evaluations_ar)

coverage_probability.append(np.sum(result.T[1]) / runs)
means_of_variance.append(np.mean(result.T[0]))
std_of_variance.append(np.std(result.T[0]))


In [ ]:
%%time
%%capture
# benchmark AR bootstrap
p = Pool()

evaluations_multiplier_iid = p.map(lambda sample: evaluate_multiplier_iid(sample, alpha=0.1, mean=mean), samples)

result = np.array(evaluations_multiplier_iid)

coverage_probability.append(np.sum(result.T[1]) / runs)
means_of_variance.append(np.mean(result.T[0]))
std_of_variance.append(np.std(result.T[0]))

In [ ]:
%%time
%%capture
# benchmark AR bootstrap
p = Pool()

evaluations_ma = p.map(lambda sample: evaluate_ma(sample, alpha=0.1, mean=mean), samples)

result = np.array(evaluations_ma)

coverage_probability.append(np.sum(result.T[1]) / runs)
means_of_variance.append(np.mean(result.T[0]))
std_of_variance.append(np.std(result.T[0]))

## Concatination result

In [ ]:
benchmark = [time_series.asymptotic_variance,
             sample_size] + means_of_variance + std_of_variance + [1 - 0.1] + coverage_probability

In [ ]:
benchmark = pd.DataFrame([benchmark], columns=pd.MultiIndex.from_tuples([
                                                             ("mean", "Asymptotic variance"),
                                                             ("Sample size", "")] +
                                                            [("mean", name,) for name in list_name_weights] +
                                                            [("std", name,) for name in list_name_weights] +
                                                            [("In confidence interval", "Confidence level")]
                                                            +
                                                            [("In confidence interval", name,) for name in
                                                             list_name_weights]
                                                            )).set_index(["Sample size"])

In [ ]:
benchmark

In [ ]:
benchmark.to_csv(f"./data/magarch-process/benchmark_{sample_size}_magarch11_2.csv")
benchmark.to_pickle(f"./data/magarch-process/benchmark_{sample_size}_magarch11_2.pkl")